In [ ]:
import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

import os



In [ ]:
DATA_PATH = '../datasets/CIFAR10/'
EPOCHES = 10
BATCH_SIZE = 64

In [ ]:
DOWNLOAD = False
if not os.path.exists(DATA_PATH):
    DOWNLOAD = True

trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
])    

train_data = datasets.CIFAR10(root=DATA_PATH, train=True, transform=trans, download=DOWNLOAD)
test_data = datasets.CIFAR10(root=DATA_PATH, train=False, transform=trans)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

# [3, 32, 32]

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            # input [batch, 3, 32, 32]
            nn.Conv2d(3, 6, 5),
            # [batch, 6, 28, 28]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # [batch, 6, 14, 14]
            nn.Conv2d(6, 16, 5),
            # [batch, 16, 10, 10]
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            # [batch, 16, 5, 5]
        )
        self.fc = nn.Sequential(
            nn.Linear(16*5*5, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10),
        )
    def forward(self, x):
        out = self.conv(x)
        out = out.view(-1, 16*5*5)
        out = self.fc(out)
        return out

model = CNN()
if torch.cuda.is_available():
    model = model.cuda()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
from IPython.core.debugger import set_trace

for epoch in range(EPOCHES):
    print('*'*10)
    print('Epoch: {}'.format(epoch))
    tloss = 0.0
    taccu = 0.0
#     set_trace()
    for i, data in enumerate(train_loader):
        img, labels = data
        x = Variable(img)
        y = Variable(labels)
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()
        outs = model(x)
        loss = criterion(outs, y)
        tloss += loss.data[0]
        pred = torch.max(outs, -1)[1]
        taccu += (pred == y).sum().data[0]
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Loss: {}'.format(tloss))
    print('Accu: {}'.format(taccu/len(train_data)))

In [ ]:
# Validation
print('Validation:')
model.eval()
tloss = 0.0
taccu = 0.0
for i, data in enumerate(test_loader):
    img, label = data
    x = Variable(img)
    y = Variable(label)
    outs = model(x)
    pred = torch.max(outs, -1)[1]
    loss = criterion(outs, y)
    tloss += loss.data[0]
    taccu += (pred == y).sum().data[0]
print('Loss: {}'.format(tloss))
print('Accu: {}'.format(taccu/len(test_data)))